# Класс `Matrix`

In [2]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

In [3]:
def matrix_product(X, Y):
    """Computes the matrix product of X and Y.

    >>> X = Matrix([[1], [2], [3]])
    >>> Y = Matrix([[4, 5, 6]])
    >>> matrix_product(X, Y)
    [[4, 5, 6], [8, 10, 12], [12, 15, 18]]
    >>> matrix_product(Y, X)
    [[32]]
    """
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    # верим, что с размерностями всё хорошо
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [4]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


In [5]:
>>> X = Matrix([[1], [2], [3]])
>>> Y = Matrix([[4, 5, 6]])
>>> matrix_product(X, Y)
[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
>>> matrix_product(Y, X)
[[32]]

[[32]]

In [42]:
%doctest_mode

Exception reporting mode: Context
Doctest mode is: OFF


# Измерение времени работы

Кажется, всё работает, но насколько быстро? Воспользуемся "магической" командой `timeit`, чтобы проверить.

In [7]:
%%timeit shape = 64, 64; X = Matrix.random(shape); Y = Matrix.random(shape)
matrix_product(X, Y)

1 loops, best of 3: 198 ms per loop


Итого: умножение двух матриц размера 64x64 занимает 0.2 секунды. Y U SO SLOW?

Определим вспомогательную функцию `bench`, которая генерирует случайные матрицы указанного размера, а затем `n_iter` раз умножает их в цикле.

In [8]:
def bench(shape=(64, 64), n_iter=16):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    for iter in range(n_iter):
        matrix_product(X, Y)    

Воспользуемся модулем `cProfile` для поиска проблемы.

In [9]:
import cProfile
source = open("faster_python_faster.py").read()
cProfile.run(source, sort="tottime")

         41646 function calls in 3.317 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       16    3.273    0.205    3.274    0.205 <string>:28(matrix_product)
     8192    0.015    0.000    0.028    0.000 random.py:170(randrange)
     8192    0.010    0.000    0.013    0.000 random.py:220(_randbelow)
     8192    0.006    0.000    0.034    0.000 random.py:214(randint)
      128    0.006    0.000    0.040    0.000 faster_python_faster.py:14(<listcomp>)
     8202    0.002    0.000    0.002    0.000 {method 'getrandbits' of '_random.Random' objects}
        1    0.001    0.001    3.315    3.315 <ipython-input-8-5f8a9b74d4e9>:1(bench)
     8192    0.001    0.000    0.001    0.000 {method 'bit_length' of 'int' objects}
        1    0.001    0.001    3.317    3.317 {built-in method exec}
       16    0.001    0.000    0.001    0.000 faster_python_faster.py:8(<listcomp>)
        2    0.000    0.000    0.040    0.020 faster_pytho

Результат предсказуемый и довольно бесполезный: >90% времени работы происходит в функции `matrix_product`. Попробуем посмотреть на неё по внимательней с помощью модуля `line_profiler`.

In [10]:
%load_ext line_profiler
%lprun -f matrix_product bench()

Заметим, что операция `list.__getitem__` не бесплатна. Переставим местами циклы `for` так, чтобы код делал меньше обращений по индексу.

In [11]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [12]:
%lprun -f matrix_product bench()

На 2 секунды быстрее, но всё равно слишком медленно: >30% времени уходит исключительно на итерацию! Поправим это.

In [13]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi, Zi = X[i], Z[i]
        for k in range(n_ycols):
            Zi[k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    return Z

In [14]:
%lprun -f matrix_product bench()

Функции `matrix_product` сильно похорошело. Но, кажется, это не предел. Попробуем снова убрать лишние обращения по индексу из самого внутреннего цикла.

In [15]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # <--
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

# Numba

Numba не работает с встроенными списками. Перепишем функцию `matrix_product` с использованием ndarray.

In [16]:
import numba
import numpy as np


@numba.jit
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

Посмотрим, что получилось.

In [17]:
shape = 64, 64
X = np.random.randint(-255, 255, shape)
Y = np.random.randint(-255, 255, shape)

%timeit -n100 jit_matrix_product(X, Y)

The slowest run took 7.33 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 335 µs per loop


# Cython

In [18]:
%load_ext cython

In [43]:
%%cython -a
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))

    
def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

In [21]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [25]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 3: 35 ms per loop


Проблема в том, что Cython не может эффективно оптимизировать работу со списками, которые могут содержать элементы различных типов, поэтому перепишем `matrix_product` с использованием *ndarray*.

In [27]:
X = np.random.randint(-255, 255, size=shape)
Y = np.random.randint(-255, 255, size=shape)

In [44]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [30]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 3: 182 ms per loop


Как же так! Стало только хуже, причём большинство кода всё ещё использует вызовы Python. Избавимся от них, про аннотировав код типами.

In [45]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [32]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 3: 189 ms per loop


К сожалению, типовые аннотации не изменили время работы, потому что тело самого вложенного цикла Cython оптимизировать не смог. Fatality-time: укажем тип элементов в *ndarray*.

In [48]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [35]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 3: 877 µs per loop


Попробуем пойти дальше и отключить проверки на выход за границы массива и переполнение целочисленных типов.

In [46]:
%%cython -a
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [37]:
%timeit -n100 cy_matrix_product(X, Y)

100 loops, best of 3: 611 µs per loop


Profit.